In [1]:
import DataFunctions
import numpy as np



Overall idea of simulation: Do pairings taking into consideration current score (win-lose rate) and their geographical location. It will use an adaptation of the Swiss Pairing System, the main difference is that instead of participants having a ELO score or rating and being paired by it; we consider an objective function on the total distance travelled and find proper pairings that minimize it. 

Tentative Objective function: $\sum (d_i)^2 + max(d_i)^2$. This function uses L2 norm and a L2 regularization term. We have to consider a regularization term as it helps us to avoid having multiple low distances and 1 large distance. For simplicity we can remove the regularization to trial the algorithm, and then regularization.

Rules per each round:
1. Split teams by their current score (#wins)
2. Pair from highest scoring group to lowest scoring group.
3. Per each group, find the group of pairs that minimize the objective function.
4. If there are odd # of teams, demote unpaired team into next scoring group. If it has been demoted before, use the next possible pairing that includes that team.
5. If there exist unpaired teams, demote them into the next scoring group. 
6. After making all possible pairings, set distance of teams that are playing together to inf.
7. For each pairing, if both teams have the same # of home games, pick one to be home and one to be away randomly. If one team has more # home games than the other, pick the team with the least # home games to be home, the other to be the away.

Things to store in db:
1. Each team 2022 stats.
2. Each team # (from 0 to 131).
3. Simulated season pairings, w-l record and h-a count.
4. Simulated games and scores.

Simulating each pairing:
1. Use 2022 stats to predict a spread and totalpts. We use those as means.
2. Using a normal distribution on the spread and totalpts with the standard deviation found on the model. We randomly generate a result with the simulated spread and totalpts.
3. We use obtained values to create an score.

In [2]:
#creating random distance matrix of size = 8
nteams = 9

m_dist = np.around(np.random.uniform(1,10,size=(nteams,nteams)),decimals=3)

for i in range(nteams):
    for j in range(nteams):
        m_dist[j,i]=m_dist[i,j]
    m_dist[i,i]=0

m_dist

array([[0.   , 6.788, 2.244, 2.021, 2.417, 9.85 , 6.086, 4.629, 4.285],
       [6.788, 0.   , 4.858, 5.37 , 9.09 , 8.252, 7.448, 2.895, 1.733],
       [2.244, 4.858, 0.   , 3.481, 2.494, 8.776, 9.597, 9.295, 8.486],
       [2.021, 5.37 , 3.481, 0.   , 3.004, 6.085, 1.993, 8.578, 7.354],
       [2.417, 9.09 , 2.494, 3.004, 0.   , 7.754, 8.941, 6.631, 3.278],
       [9.85 , 8.252, 8.776, 6.085, 7.754, 0.   , 6.64 , 6.646, 1.713],
       [6.086, 7.448, 9.597, 1.993, 8.941, 6.64 , 0.   , 6.429, 1.202],
       [4.629, 2.895, 9.295, 8.578, 6.631, 6.646, 6.429, 0.   , 1.547],
       [4.285, 1.733, 8.486, 7.354, 3.278, 1.713, 1.202, 1.547, 0.   ]])

In [3]:
#storing wins per team.
curr_data = np.zeros(shape=(nteams,2),dtype=int)
curr_data[:,0] = np.arange(nteams)
curr_data

array([[0, 0],
       [1, 0],
       [2, 0],
       [3, 0],
       [4, 0],
       [5, 0],
       [6, 0],
       [7, 0],
       [8, 0]])

In [4]:
#splitting in groups
sim1group0 = np.array([x[0].astype(int) for x in curr_data if x[1]==0])
# sim1group1 = [[x[0].astype(int),0] for x in curr_data if x[1]==1]
sim1group0

array([0, 1, 2, 3, 4, 5, 6, 7, 8])

We cannot find all pairs and run for all of them. That operation has time complexity O(n!) and for 130ish teams is not feasible as it is about $10^{220}$. We need to use another approach.<br>
Fastest approach is done by sorting all possible distances, then picking the suitable one and remove unsuitables from the list. We only need to sort once since at the beginning all teams have to play. We should also count the # of possible matches and use that to do the sorting as well.<br><br>

Algorithm procedure using networkx and Blossom algorithm (also known as Edmonds' algorithm)

In [5]:
import networkx as nx

In [6]:
L = []
Gs1g0 = nx.Graph()

for i in sim1group0:
    for j in sim1group0:
        if i<j: 
            L.append((i,j,m_dist[i,j]))

In [7]:
Gs1g0.add_weighted_edges_from(L)

In [8]:
Gs1g0.edges.data("weight")

EdgeDataView([(0, 1, 6.788), (0, 2, 2.244), (0, 3, 2.021), (0, 4, 2.417), (0, 5, 9.85), (0, 6, 6.086), (0, 7, 4.629), (0, 8, 4.285), (1, 2, 4.858), (1, 3, 5.37), (1, 4, 9.09), (1, 5, 8.252), (1, 6, 7.448), (1, 7, 2.895), (1, 8, 1.733), (2, 3, 3.481), (2, 4, 2.494), (2, 5, 8.776), (2, 6, 9.597), (2, 7, 9.295), (2, 8, 8.486), (3, 4, 3.004), (3, 5, 6.085), (3, 6, 1.993), (3, 7, 8.578), (3, 8, 7.354), (4, 5, 7.754), (4, 6, 8.941), (4, 7, 6.631), (4, 8, 3.278), (5, 6, 6.64), (5, 7, 6.646), (5, 8, 1.713), (6, 7, 6.429), (6, 8, 1.202), (7, 8, 1.547)])

In [9]:
matchings1g0 = nx.algorithms.matching.min_weight_matching(Gs1g0)

In [14]:
list(matchings1g0)

[(1, 7), (2, 4), (0, 3), (8, 6)]

In [13]:
#checking 
m_dist[0,3],m_dist[1,7],m_dist[2,4],m_dist[8,6]

(2.021, 2.895, 2.494, 1.202)

In [18]:
LS = sorted(np.array(L)[:,2])
LS

[1.202,
 1.547,
 1.713,
 1.733,
 1.993,
 2.021,
 2.244,
 2.417,
 2.494,
 2.895,
 3.004,
 3.278,
 3.481,
 4.285,
 4.629,
 4.858,
 5.37,
 6.085,
 6.086,
 6.429,
 6.631,
 6.64,
 6.646,
 6.788,
 7.354,
 7.448,
 7.754,
 8.252,
 8.486,
 8.578,
 8.776,
 8.941,
 9.09,
 9.295,
 9.597,
 9.85]